In [1]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path=r"C:\Users\bhushan\Desktop\dev1\semantic_st25\nco_data_with_desc_500_new.csv",
    content_columns=["description"],
    metadata_columns=["occupation_title", "nco_2015", "nco_2004", "division", "subdivision", "group", "family", "sno"]
)

documents = loader.load()

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [4]:
from langchain_community.vectorstores.faiss import FAISS

texts = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]

faiss_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

In [18]:
import pandas as pd

query = "jobs in banking sector"
rows = []

results = faiss_store.similarity_search_with_score(query, k=50)

for doc, score in results:
    row = doc.metadata.copy()
    row["similarity_score"] = score
    rows.append(row)

df = pd.DataFrame(rows)
print(df[[
    "sno", "similarity_score", "occupation_title", "nco_2015", "nco_2004",
    "division"
]].to_string(index=False))

sno  similarity_score                                                                 occupation_title  nco_2015 nco_2004      division
 32          1.024390                                                                   Director, Bank 1120.0700  1212.10      Managers
  2          1.044280                                                            General Manager, Bank 1211.0100  1307.10      Managers
165          1.103403                                                                    Manager, Bank 1346.0100  1227.10      Managers
167          1.137011                           Financial and Insurance Service Branch Managers, Other 1346.9900  1227.90      Managers
223          1.223267                               Mathematicians, Actuaries and Statisticians, Other 2120.9900  2122.90 Professionals
  1          1.341756                                                         Finance Managers, Others 1211.9900   1231.9      Managers
 77          1.380076                           